### Set Project Root

In [1]:
from constants import set_cwd_to_project_root

set_cwd_to_project_root()

PAIR = "BTC/USDT:USDT"

### Initialize Datahandler

In [2]:
# Load data using values set above
from freqtrade.data.history import get_datahandler

from freqtrade.enums import CandleType
from constants import DATA_DIR

data_handler = get_datahandler(
    datadir=DATA_DIR / "binance",
    data_format="feather",
)

### Load Trades

In [6]:
trades = data_handler.trades_load(PAIR, CandleType.FUTURES).set_index("date")

trades.head()

,timestamp,id,type,side,price,amount,cost
date,,,,,,,
2024-09-01 00:00:00+00:00,1725148800000,2311244041,None,buy,58941.9,0.224,13202.9856
2024-09-01 00:00:02.444000+00:00,1725148802444,2311244042,None,sell,58941.8,0.115,6778.3070
2024-09-01 00:00:03.431000+00:00,1725148803431,2311244043,None,sell,58941.8,0.156,9194.9208
2024-09-01 00:00:03.473000+00:00,1725148803473,2311244044,None,buy,58941.9,0.008,471.5352
2024-09-01 00:00:03.475000+00:00,1725148803475,2311244045,None,sell,58941.8,5.366,316281.6988


### Load Candles

In [8]:
candles = data_handler.ohlcv_load(PAIR, "5m", CandleType.FUTURES).set_index("date")

candles = candles[trades.index.min():trades.index.max()]

candles.head()

,open,high,low,close,volume
date,,,,,
2024-09-01 00:00:00+00:00,58941.9,58951.4,58925.2,58951.4,95.907
2024-09-01 00:05:00+00:00,58951.4,59011.6,58943.7,59004.5,202.985
2024-09-01 00:10:00+00:00,59004.6,59033.0,58990.1,59008.9,227.436
2024-09-01 00:15:00+00:00,59009.0,59052.7,58992.8,58992.8,285.417
2024-09-01 00:20:00+00:00,58992.8,58998.1,58960.0,58960.0,156.475


### Merge Trades and Candles

In [11]:
from freqtrade.data.converter import populate_dataframe_with_trades
from collections import OrderedDict

# Initialize an empty OrderedDict for cached_grouped_trades
cached_grouped_trades = OrderedDict()

dataframe, cached_grouped_trades = populate_dataframe_with_trades(
    cached_grouped_trades=cached_grouped_trades,
    config={
        "timeframe": "5m", 
        "orderflow": {
            "cache_size": 1000,
            "max_candles": 10000,
            "scale": 0.5,
            "stacked_imbalance_range": 3,
            "imbalance_volume": 1,
            "imbalance_ratio": 3
        }
    },
    dataframe=candles.reset_index(),
    trades=trades.reset_index()
)

del candles
del trades

dataframe.head()

### Populate indicators

In [ ]:
from pandas import DataFrame

dataframe["day"] = dataframe["date"].dt.date

dataframe.set_index("date", inplace=True)

def _calculations(group: DataFrame) -> DataFrame:
    # Calculate CVD and related indicators
    group["cvd"] = group["delta"].cumsum()

    return group

dataframe = dataframe.groupby("day").apply(_calculations).reset_index(level=0, drop=True)

dataframe.head()

C:\Users\j10st\AppData\Local\Temp\ipykernel_28648\662854625.py:13: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,open,high,low,close,volume,trades,orderflow,imbalances,stacked_imbalances_bid,stacked_imbalances_ask,...,ask,delta,total_trades,day,cvd,price_change,cvd_change,price_cvd_diff,divergence,ml_divergence
date,,,,,,,,,,,,,,,,,,,,,
2024-09-22 22:50:00+00:00,63628.1,63900.0,63541.9,63827.7,3687.681,"[{'timestamp': 1727045400188, 'id': '233836447...","{63542.0: {'bid': 12, 'ask': 8, 'delta': -0.44...","{63542.0: {'bid_imbalance': True, 'ask_imbalan...",63557.0,63832.5,...,2391.178,1094.675,14871.0,2024-09-22,1094.675,0.003137,0.404781,60028.663,0,0
2024-09-23 03:25:00+00:00,64580.7,64724.8,64520.6,64630.7,3204.886,"[{'timestamp': 1727061900033, 'id': '233875328...","{64520.5: {'bid': 2, 'ask': 0, 'delta': -0.89,...","{64520.5: {'bid_imbalance': False, 'ask_imbala...",64526.0,64723.5,...,1734.452,263.912,13178.0,2024-09-23,263.912,0.000774,0.109193,61949.861,0,0
2024-09-23 04:20:00+00:00,64153.0,64211.8,63972.3,63995.2,1983.184,"[{'timestamp': 1727065200018, 'id': '233883171...","{63972.5: {'bid': 5, 'ask': 0, 'delta': -0.9, ...","{63972.5: {'bid_imbalance': False, 'ask_imbala...",63977.0,64188.5,...,771.461,-440.263,6851.0,2024-09-23,-176.351,-0.002458,-0.328112,63093.655,0,0
2024-09-23 04:40:00+00:00,63901.1,63934.5,63827.0,63854.6,633.762,"[{'timestamp': 1727066400042, 'id': '233886091...","{63827.0: {'bid': 3, 'ask': 0, 'delta': -2.733...","{63827.0: {'bid_imbalance': True, 'ask_imbalan...",63833.0,63930.5,...,272.012,-89.738,4010.0,2024-09-23,-266.089,-0.000728,-0.083947,62875.360,0,0
2024-09-23 14:15:00+00:00,63441.2,63778.7,63400.0,63648.0,3850.686,"[{'timestamp': 1727100900097, 'id': '233923750...","{63400.0: {'bid': 4, 'ask': 3, 'delta': 0.0959...","{63400.0: {'bid_imbalance': False, 'ask_imbala...",63416.5,63733.5,...,2208.628,566.626,15438.0,2024-09-23,300.537,0.003260,7.427751,63005.089,0,0


### Imports

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Add features for model training

In [ ]:
dataframe['price_change'] = dataframe['close'].pct_change()
dataframe['cvd_change'] = dataframe['cvd'].pct_change()
dataframe['price_cvd_diff'] = dataframe['close'] - dataframe['cvd']

### Define target variable (1 for divergence, 0 for no divergence)

In [ ]:
dataframe['divergence'] = (dataframe['price_change'] * dataframe['cvd_change'] < 0).astype(int)

# Drop NaN values
dataframe.dropna(inplace=True)

### Features and target

In [ ]:
X = dataframe[['price_change', 'cvd_change', 'price_cvd_diff']]
y = dataframe['divergence']

### Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train the model

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

### Predict and evaluate

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

f"Model Accuracy: {accuracy:.2f}"

### Strategy Integration

In [ ]:
def detect_divergence_with_ml(dataframe: pd.DataFrame) -> pd.Series:
    features = dataframe[['price_change', 'cvd_change', 'price_cvd_diff']].fillna(0)
    predictions = model.predict(features)
    return pd.Series(predictions, index=dataframe.index)

# Integrate into your strategy
dataframe['ml_divergence'] = detect_divergence_with_ml(dataframe)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create the candlestick chart
fig = go.Figure(
    data=[
        go.Candlestick(
            x=dataframe.index,
            open=dataframe['open'],
            high=dataframe['high'],
            low=dataframe['low'],
            close=dataframe['close']
        )
])

# Add ML divergence markers
divergence_points = dataframe[dataframe['ml_divergence'] == 1]

fig.add_trace(go.Scatter(
    x=divergence_points.index,
    y=divergence_points['close'],
    mode='markers',
    marker=dict(
        size=10,
        symbol='star',
        color='red',
        line=dict(width=2, color='DarkSlateGrey')
    ),
    name='ML Divergence'
))

# Add CVD subplot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.7, 0.3])

# Add candlestick chart to the first subplot
fig.add_trace(
    go.Candlestick(
        x=dataframe.index,
        open=dataframe['open'],
        high=dataframe['high'],
        low=dataframe['low'],
        close=dataframe['close'],
        name='Candlesticks'
    ),
    row=1, col=1
)

# Add ML divergence markers to the first subplot
fig.add_trace(
    go.Scatter(
        x=divergence_points.index,
        y=divergence_points['close'],
        mode='markers',
        marker=dict(
            size=10,
            symbol='star',
            color='red',
            line=dict(width=2, color='DarkSlateGrey')
        ),
        name='ML Divergence'
    ),
    row=1, col=1
)

# Add CVD to the second subplot
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe['cvd'],
        mode='lines',
        name='CVD'
    ),
    row=2, col=1
)

# Update layout for both subplots
fig.update_layout(
    title=f'{PAIR} OHLCV Chart with Pivot Points and CVD',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=800  # Increase the overall height to accommodate the subplot
)

# Update y-axis labels
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="CVD", row=2, col=1)


# Update layout
fig.update_layout(
    title=f'{PAIR} OHLCV Chart with Pivot Points',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the plot
fig.show()